# Library

In [ ]:
%pip install pyvi
%pip install underthesea

In [ ]:
import os
import re
import pyvi
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from pyvi import ViTokenizer
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.python.keras import backend as K
from tensorflow.keras.activations import softmax
from tensorflow.python.keras.layers import Layer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, Concatenate, Dropout
from underthesea import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


False

# Read data

In [ ]:
def read_file(filename):
    lines = open(filename, 'r', encoding='utf8').readlines()
    return lines
def read_data(data,Q,A):
    Replace_element=['',' ']
    for line in data:
        count=0
        if line in Replace_element:
            continue
        for sentence in line.strip().split(' '):
            count+=1
            if sentence in Replace_element:
                if count ==1:
                    break
                if count==2:
                    Q=Q[:-1]
            if count==1:
                Q.append(sentence)
            if count==2:
                A.append(sentence.replace(' ',''))
                if '?' in sentence:
                    A[-1] = A[-1].replace('?','')


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
dt = pd.read_csv('ChatBotData.csv', encoding='utf-8')
dt.head()

,Unnamed: 0,user_a,user_b
0,0,bạn tên gì?,Mình tên Trường <eos> còn bạn tên gì?
1,1,Tui tên Nam <eos> Bạn học ở đâu?,tui học ở TDTU nè <eos> còn bạn học ở trường n...
2,2,tui học ở TDTU luôn <eos> Bạn học ngành gì,mình học Công Nghệ Thông Tin
3,3,Chuyên ngành gì vậy bạn ?,mình học bên Kỹ Thuật Phần Mềm á <eos> còn bạn...
4,4,mình học Quản Trị Kinh Doanh <eos> Bạn là ngườ...,Mình là người Đồng tháp <eos> bạn biết Đồng Th...


In [ ]:
Q = dt['user_a']
A = dt['user_b']
print(Q.head())
print(A.head())
data = pd.DataFrame()
data['question'] = Q
data['answer'] = A

0                                          bạn tên gì?
1                     Tui tên Nam <eos> Bạn học ở đâu?
2           tui học ở TDTU luôn <eos> Bạn học ngành gì
3                            Chuyên ngành gì vậy bạn ?
4    mình học Quản Trị Kinh Doanh <eos> Bạn là ngườ...
Name: user_a, dtype: object
0                Mình tên Trường <eos> còn bạn tên gì?
1    tui học ở TDTU nè <eos> còn bạn học ở trường n...
2                         mình học Công Nghệ Thông Tin
3    mình học bên Kỹ Thuật Phần Mềm á <eos> còn bạn...
4    Mình là người Đồng tháp <eos> bạn biết Đồng Th...
Name: user_b, dtype: object


 Processing word segmentation

---



In [ ]:
temp = []
for question in data['question']:
  temp.append(word_tokenize(question))
#print(temp)

In [ ]:
smoothie = SmoothingFunction().method4
def predict(candidate):
  candidate = word_tokenize(candidate)
  list_core = []
  max = {'bleu_core': 0}
  for i in range(len(temp)):
    tmp = {}
    tmp['question'] = temp[i]
    tmp['bleu_core'] = sentence_bleu([temp[i]], candidate,smoothing_function=smoothie)
    tmp['answer'] = A[i]
    if(tmp['bleu_core'] > max['bleu_core']):
      max = tmp
  if(max['bleu_core'] == 0):
    return 'Bạn vui lòng hỏi câu khác!!'
  return max['answer']

In [ ]:
s=''
while s!='Tạm biệt':
  s=input('You chat : ')
  if s.lower() == 'tạm biệt':
    print('Bot chat: Tạm biệt. Chúc bạn một ngày vui vẻ!')
    break
  print('Bot chat:', predict(s))

Bot chat: Ai vậy ạ?
Bot chat: A, có phải Hiếu PC đấy không. Tao nè, Vinh Khuất quê Sơn La nè
Bot chat: Mình tên Trường <eos> còn bạn tên gì?
Bot chat: Thế minh học trường nào thế?
Bot chat: à vậy mình cũng học gần trường của bạn đấy
Bot chat: nan
Bot chat: Chưa.
Bot chat: Bạn vui lòng hỏi câu khác!!
Bot chat: Tạm biệt. Chúc bạn một ngày vui vẻ!


In [ ]:
from gensim.models import Word2Vec

modelw2v = Word2Vec(sentences=temp, vector_size=100, window=5, sg=0, min_count=1)
wordlist = list(modelw2v.wv.key_to_index)
wordlist[:5]

['?', ',', 'không', '.', 'bạn']

In [ ]:
def predict_w2v(input):
  input = word_tokenize(input)
  list_core = []
  min = {'dst': 100}
  for i in range(1, len(temp)):
    #tính khoảng cách giữa 2 câu
    dst = modelw2v.wv.wmdistance(temp[i], input)
    tmp = {}
    tmp['question'] = temp[i]
    tmp['dst'] = dst
    tmp['answer'] = A[i]
    #tìm khoảng cách bé nhất
    if(tmp['dst'] < min['dst']):
      min = tmp
  return min['answer']

In [ ]:
s=''
while s!='Tạm biệt':
    s=input('You chat : ')
    if s.lower() == 'tạm biệt':
      print('Bot chat: Tạm biệt. Chúc bạn một ngày vui vẻ!')
      break
    print('Bot chat:', predict_w2v(s))

Bot chat: Xin chào
Bot chat: Mình tên Khánh <eos> Còn bạn tên gì?
Bot chat: Mình ten Duy <eos> Còn bạn tên gì?
Bot chat: Tớ học đại học Tôn Đức Thắng
Bot chat: Bạn học ngành gì ?
Bot chat: nan
Bot chat: tớ học Không giỏi
Bot chat: Thế bạn khoa nào?
Bot chat: nan
Bot chat: À môn đó tui chưa học qua
Bot chat: nan
Bot chat: Mình thích hãng phim Marvel nhất
Bot chat: Thế bạn khoa nào?
Bot chat: crush bao lâu rồi?
Bot chat: nan
Bot chat: Tạm biệt. Chúc bạn một ngày vui vẻ!
